# Improvements after seeing others' Notebooks

Inspiration largely from Chong Zhen Jie:
- notebook - https://www.kaggle.com/code/chongzhenjie/ecuador-store-sales-global-forecasting-lightgbm/notebook#3.-Model-Training
- eda takeaways (not used here)
    * missed zero vals (leading, trailing 0s, number of 0s)
    * holidays (filtering for region, work days, etc.)
    * store clustering

- modelling takeaways
    * interpolate missing dates (like jan. 01, christmas)
    * dif model for each family
    * different samples of training data --> ensemble models

## What are the main improvements I can make?

- features
    * oil, oil rolling
    * transactions
    * store cluster, type, etc.

    * holidays merged (city, state, national, work) - or maybe we can delete holidays

- modelling
    * train dif model per family
    * zero out predictions if there are some # of trailing 0s
    * train models on dif train periods (can also exclude before 2015), then ensemble
    * try weighted lgbm highlighting end of aug (own idea, not sure if itll help at all)

In [71]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error
import lightgbm as lgb
from itertools import product
import re

In [59]:
train_df = pd.read_csv('data/train.csv', parse_dates=['date']).drop(columns='id')
test_df = pd.read_csv('data/test.csv', parse_dates=['date']).drop(columns='id')
transactions_df = pd.read_csv('data/transactions.csv', parse_dates=['date'])
oil_df = pd.read_csv('data/oil.csv', parse_dates=['date'])
holidays_df = pd.read_csv('data/holidays_events.csv', parse_dates=['date'])
stores_df = pd.read_csv('data/stores.csv')

## 1. Preprocessing (fix missing vals)

In [4]:
# Fix missing dates / NaNs in oil df
start_day = oil_df['date'].min()
end_day = oil_df['date'].max()
date_range = pd.date_range(start_day, end_day)
print(f"Days in Oil Range: {start_day.strftime('%Y-%m-%d')} to {end_day.strftime('%Y-%m-%d')}, {len(date_range)} days")
print(f"Days in Actual oil df: {len(oil_df['date'])}")

oil_all_days = oil_df.merge(pd.DataFrame({'date': date_range}), how='outer', on='date')
interpolated_oil = oil_all_days.interpolate(method='linear', limit_direction='both')
print(f"After interpolating, {interpolated_oil.isna().sum().sum()} missing values")

Days in Oil Range: 2013-01-01 to 2017-08-31, 1704 days
Days in Actual oil df: 1218
After interpolating, 0 missing values


In [56]:
# Fix missing values in train
start_day = train_df['date'].min()
end_day = train_df['date'].max()
date_range = pd.date_range(start_day, end_day)
print(f"Days in Train Range: {start_day.strftime('%Y-%m-%d')} to {end_day.strftime('%Y-%m-%d')}, {len(date_range)} days")
print(f"Days in Actual Train df: {len(train_df['date'].unique())}")
print(f"Missing days:", date_range.difference(train_df['date'].unique()))
print()

# Fill in Christmas
stores = train_df['store_nbr'].unique()
families = train_df['family'].unique()
missing_days = date_range.difference(train_df['date'].unique())
missing_index = pd.DataFrame(data=list(product(missing_days, stores, families)), columns=['date', 'store_nbr', 'family'])
missing_index = missing_index.reindex(columns=train_df.columns)

all_days_df = pd.concat([train_df, missing_index]).sort_values(by=['date', 'store_nbr', 'family'])
all_days_df.head()

# Check that all values present
print("# Stores:", len(stores))
print("# Families:", len(families))
print("# Days:", len(all_days_df['date'].unique()))
print("Length of new df:", len(all_days_df))
print("Stores * Families * Days:", len(stores) * len(families) * len(all_days_df['date'].unique()))
print("Missing values:", all_days_df['sales'].isna().sum())
print("Expected missing values:", len(stores) * len(families) * len(missing_days))

# Then interpolate between days within the same STORE and FAMILY
interpolated = all_days_df.groupby(by=['store_nbr', 'family']).apply(lambda col : col.interpolate(method='linear'), include_groups=False)
interpolated = interpolated.reset_index().drop(columns='level_2')
interpolated[interpolated['date'].isin(missing_days)]

Days in Train Range: 2013-01-01 to 2017-08-15, 1688 days
Days in Actual Train df: 1684
Missing days: DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)

# Stores: 54
# Families: 33
# Days: 1688
Length of new df: 3008016
Stores * Families * Days: 3008016
Missing values: 7128
Expected missing values: 7128


,store_nbr,family,date,sales,onpromotion
358,1,AUTOMOTIVE,2013-12-25,1.5,0.0
723,1,AUTOMOTIVE,2014-12-25,3.5,0.0
1088,1,AUTOMOTIVE,2015-12-25,5.5,0.0
1454,1,AUTOMOTIVE,2016-12-25,3.0,0.0
2046,1,BABY CARE,2013-12-25,0.0,0.0
...,...,...,...,...,...
3006094,54,SCHOOL AND OFFICE SUPPLIES,2016-12-25,0.0,0.0
3006686,54,SEAFOOD,2013-12-25,0.5,0.0
3007051,54,SEAFOOD,2014-12-25,0.0,0.0
3007416,54,SEAFOOD,2015-12-25,1.0,0.5


In [72]:
# Process the holidays

# 1. lowercase all columns
processed_holidays = holidays_df.copy()
processed_holidays = processed_holidays.apply(lambda col : col.str.lower() if col.dtype == 'object' else col)


# 2. remove cities
to_remove = ['de', 'del', 'el', 'la']
to_remove.extend(processed_holidays['locale_name'].unique())
remove_regex = r'\b(?:' + '|'.join(map(re.escape, to_remove)) + r')\b'


processed_holidays['description'] = processed_holidays['description'].replace(to_replace=remove_regex, value='', regex=True)
processed_holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,holiday,local,manta,fundacion,false
1,2012-04-01,holiday,regional,cotopaxi,provincializacion,false
2,2012-04-12,holiday,local,cuenca,fundacion,false
3,2012-04-14,holiday,local,libertad,cantonizacion,false
4,2012-04-21,holiday,local,riobamba,cantonizacion,false
...,...,...,...,...,...,...
345,2017-12-22,additional,national,ecuador,navidad-3,false
346,2017-12-23,additional,national,ecuador,navidad-2,false
347,2017-12-24,additional,national,ecuador,navidad-1,false
348,2017-12-25,holiday,national,ecuador,navidad,false


In [73]:
processed_holidays['description'].unique()

array(['fundacion  ', 'provincializacion  ', 'cantonizacion  ',
       'cantonizacion   carmen', 'primer grito  independencia',
       'independencia  ', 'traslado independencia  ', 'dia  difuntos',
       'provincializacion ', 'fundacion  -1', 'navidad-4', 'navidad-3',
       'navidad-2', 'puente navidad', 'navidad-1', 'navidad', 'navidad+1',
       'puente primer dia  ano', 'primer dia  ano-1', 'primer dia  ano',
       'recupero puente navidad', 'recupero puente primer dia  ano',
       'carnaval', 'viernes santo', 'dia  trabajo', 'dia   madre-1',
       'dia   madre', 'batalla  pichincha',
       'inauguracion mundial  futbol brasil',
       'mundial  futbol brasil: -suiza',
       'mundial  futbol brasil: -honduras',
       'mundial  futbol brasil: -francia',
       'mundial  futbol brasil: octavos  final',
       'mundial  futbol brasil: cuartos  final',
       'mundial  futbol brasil: semifinales',
       'mundial  futbol brasil: tercer y cuarto lugar',
       'mundial  futbol b